In [5]:
import pandas as pd
import sqlite3


df = pd.read_csv('/data.csv', encoding='latin1')


conn = sqlite3.connect(':memory:')
df.to_sql('ecommerce', conn, index=False, if_exists='replace')
cursor = conn.cursor()


print("SELECT * FROM ecommerce LIMIT 5:")
print(pd.read_sql("SELECT * FROM ecommerce LIMIT 5;", conn))


print("\nItems with UnitPrice > 10:")
print(pd.read_sql("SELECT * FROM ecommerce WHERE UnitPrice > 10 LIMIT 5;", conn))

print("\nTop 5 expensive items:")
print(pd.read_sql("SELECT Description, UnitPrice FROM ecommerce ORDER BY UnitPrice DESC LIMIT 5;", conn))

print("\nAverage UnitPrice by Country:")
print(pd.read_sql("SELECT Country, AVG(UnitPrice) as AvgPrice FROM ecommerce GROUP BY Country LIMIT 5;", conn))

query5 = """
SELECT a.InvoiceNo, a.Description, b.Quantity
FROM ecommerce a
INNER JOIN ecommerce b ON a.StockCode = b.StockCode
LIMIT 5;
"""
print("\nSelf JOIN example (StockCode):")
print(pd.read_sql(query5, conn))

query6 = """
SELECT Description, UnitPrice
FROM ecommerce
WHERE UnitPrice > (SELECT AVG(UnitPrice) FROM ecommerce)
LIMIT 5;
"""
print("\nItems priced above average:")
print(pd.read_sql(query6, conn))

cursor.execute("""
CREATE VIEW high_value_items AS
SELECT StockCode, Description, UnitPrice
FROM ecommerce
WHERE UnitPrice > 20;
""")
print("\nView created: high_value_items")

print("\nItems from view (UnitPrice > 20):")
print(pd.read_sql("SELECT * FROM high_value_items LIMIT 5;", conn))

cursor.execute("CREATE INDEX idx_stockcode ON ecommerce(StockCode);")
print("\nIndex created on StockCode")

conn.close()


SELECT * FROM ecommerce LIMIT 5:
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

      InvoiceDate  UnitPrice  CustomerID         Country  
0  12/1/2010 8:26       2.55     17850.0  United Kingdom  
1  12/1/2010 8:26       3.39     17850.0  United Kingdom  
2  12/1/2010 8:26       2.75     17850.0  United Kingdom  
3  12/1/2010 8:26       3.39     17850.0  United Kingdom  
4  12/1/2010 8:26       3.39     17850.0  United Kingdom  

Items with UnitPrice > 10:
  InvoiceNo StockCode                      Description  Quantity  \
0    536370      POST                          POSTAGE         3   
1    536374     